# MMD自動トレースキット準備

このノートブックでは、MMD自動トレースの準備と実行を行います。

## ランタイムをGPUに変更

ヘッダの "ランタイム"　＞　"ランタイムのタイプを変更"　＞　"GPU"　を選択して下さい。

変更できたら、下のセルを実行して下さい。

In [0]:
! nvcc --version
! nvidia-smi

下記のように表示されていたら成功です。

![GPU変更成功](https://drive.google.com/uc?export=view&id=17CG697kiTLkwVOdH1wg2W3MSB-hyi9u5)

下記のように表示されていたら、ランタイムの変更に失敗しているので、再度確認してください。

![GPU切り替え失敗](https://drive.google.com/uc?export=view&id=1tufSuT7ocWxv3HkrmA5kwlemhu0gv6Je)

## 準備開始

画面左上の「＞」をクリックして下さい。目次が開きます。

![目次](https://drive.google.com/uc?export=view&id=1x8AdFNmsIQPrtYptBf_NXPRNBJF8ON8z)

目次の「MMD自動トレースキット実行」を選択して下さい。

![実行選択](https://drive.google.com/uc?export=view&id=1YaSrtio06mmu94RUkU5dSolNtFqPiAnV)

ヘッダの　"ランタイム"　＞　"より前のセルを実行"　を選択すると、準備が開始されます。

![すべてのセルを実行](https://drive.google.com/uc?export=view&id=1rIbVI_Qyjs8idEzkIR0IBIx6Dbp3te08)

画面の一番下に、以下のように出力されれば、完了です。

![処理成功](https://drive.google.com/uc?export=view&id=1D21xezv6QN0RQF5ZU_LR7PRnOk0Dw4Sc)

大体40分くらいかかります。

![処理失敗](https://drive.google.com/uc?export=view&id=1t-immeF3Ji1_GBNatZOG1C07j42de4Rq)

最後の行に、「No such file or directory」と出力されていたら失敗です。

解決方法が分からない場合、私までご連絡ください。

## 環境設定

In [0]:
# 処理時間計測のための開始時間
import time
start_time = time.time()

In [0]:
# Openposeバージョンタグ
ver_openpose = "v1.4.0"

In [0]:
# キットバージョンタグ
ver_tag = "ver1.00"

## cmake

In [0]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.13.4/cmake-3.13.4.tar.gz"
! tar xf cmake-3.13.4.tar.gz
! cd cmake-3.13.4 && ./configure && make && sudo make install

## Openpose

In [0]:
# ライブラリのインストール

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python2 libs
! sudo apt-get --assume-yes install python-setuptools python-dev build-essential
! sudo easy_install pip
! sudo -H pip install --upgrade numpy protobuf opencv-python
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCV 2.4 -> Added as option
# # sudo apt-get --assume-yes install libopencv-dev
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [0]:
#  Openpose の clone
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 

In [0]:
#  Openpose の モデルデータDL
! cd openpose/models && ./getModels.sh

In [0]:
# Openpose の ビルド
! sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
! cd openpose && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc` # example demo usage

## FCRN-DepthPrediction-vmd

In [0]:
# FCRN-DepthPrediction-vmd の clone
! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/FCRN-DepthPrediction-vmd.git

In [0]:
# FCRN-DepthPrediction-vmd の モデルデータDL

# モデルデータ の導入
! mkdir -p ./FCRN-DepthPrediction-vmd/tensorflow/data

# モデルデータのダウンロード
! cd  ./FCRN-DepthPrediction-vmd/tensorflow/data && wget -c "http://campar.in.tum.de/files/rupprecht/depthpred/NYU_FCRN-checkpoint.zip" && unzip NYU_FCRN-checkpoint.zip

## 3d-pose-baseline-vmd

In [0]:
# 3d-pose-baseline-vmd の clone
! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/3d-pose-baseline-vmd.git

In [0]:
# 3d-pose-baseline-vmd の Human3.6MモデルデータDL

# Human3.6Mモデルデータ の導入
! mkdir -p ./3d-pose-baseline-vmd/data

# Human3.6Mモデルデータのダウンロード
! cd  ./3d-pose-baseline-vmd/data && wget -c "https://www.dropbox.com/s/e35qv3n6zlkouki/h36m.zip" && unzip h36m.zip

In [0]:
# 3d-pose-baseline-vmd の 学習データDL

# 3d-pose-baseline用学習データ の導入
! mkdir -p ./3d-pose-baseline-vmd/experiments

# 3d-pose-baseline用学習データのダウンロード
file_id = "1v7ccpms3ZR8ExWWwVfcSpjMsGscDYH7_"
file_name = "experiments.zip"
! cd  ./3d-pose-baseline-vmd && curl -sc ./cookie "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
code = "$(awk '/_warning_/ {print $NF}' ./cookie)"  
! cd  ./3d-pose-baseline-vmd && curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$code&id=$file_id" -o "$file_name"
! cd  ./3d-pose-baseline-vmd && unzip experiments.zip

## VMD-3d-pose-baseline-multi

In [0]:
# VMD-3d-pose-baseline-multi の clone

! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/VMD-3d-pose-baseline-multi.git

In [0]:
# VMD-3d-pose-baseline-multi のライブラリ

! sudo apt-get install python3-pyqt5  
! sudo apt-get install pyqt5-dev-tools
! sudo apt-get install qttools5-dev-tools

## 準備結果確認

In [0]:
# サンプルの実行確認
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --write_json ./output/ --display 0  --write_video ./output/openpose.avi

In [0]:
import time

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■すべての処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo "Openpose実行結果"

! ls -l ./openpose/output/openpose.avi

# MMD自動トレースキット実行

ここからは、キットを実際に実行していきます。

## 入力映像ファイルアップロード

処理したい映像ファイルを、準備してください。

 - FPSは、30fpsに固定してください。
 - 解像度は、1280x720にしてください。
 - 複数人数のトレースの場合、画面上から人物が消えると、並び順が取得できなくなるので、できるだけ取得したい人数分全員が映っているようにしてください。

準備が出来たら、Googleドライブの "autotrace" フォルダ に、半角英数字のファイル名でアップロードしてください。

アップロードが終わったら、下のフォームにファイル名を入力してください。

フォームの値を変更すると、左側のコード欄の値が変わりますが、そのままで大丈夫です。

In [0]:
#@markdown ### 入力映像ファイル
#@markdown 解析対象となる映像のファイルの名前を入力して下さい。
#@markdown Googleドライブの "autotrace" フォルダ の直下に置いてください。
input_video_name = "input.mp4"  #@param {type: "string"}

!echo "入力映像ファイルは" "$input_video_name" "で合っていますか？"
!echo "問題なければ、次へ進んで下さい。"
!echo "違う場合、ファイル名欄をもう一度確認してください。"

フォーム名が入力できたら、下のセルを実行して下さい。

In [0]:
from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

! echo "autotraceフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

# 入力動画ファイル
input_video = base_path + "/"+ input_video_name

print("ファイル名: ", os.path.basename(input_video))
print("ファイルサイズ: ", os.path.getsize(input_video))

最後のファイル名とファイルサイズが取得できていたら成功です。

![input.mp4](https://drive.google.com/uc?export=view&id=174pRGkUR7PH5hDyzfsmYmzOrmftQxrbP)

## パラメーター設定

In [0]:

#@markdown ---

#@markdown ### 解析を開始するフレーム
#@markdown 解析を開始するフレームNoを入力して下さい。(0始まり)
#@markdown 最初にロゴが表示されている等、人体が正確にトレースできない場合に、冒頭のフレームをスキップできます。
frame_first = 0  #@param {type: "number"}

#@markdown ---

#@markdown ### 映像に映っている最大人数
#@markdown 映像から取得したい人数を入力して下さい。
#@markdown 常にこの人数分映っているように、映像データを加工しておいてください。
number_people_max = 1  #@param {type: "number"}

#@markdown --- 

#@markdown ### 反転フレームリスト
#@markdown Openposeが誤認識して反転しているフレーム番号(0始まり)を指定してください。
#@markdown ここで指定された番号のフレームに対して、反転判定を行い、反転認定された場合、関節位置が反転されます。
#@markdown カンマで複数件指定可能です。また、ハイフンで範囲が指定可能です。
#@markdown 例）4,10-12　…　4,10,11,12 が反転判定対象フレームとなります。
order_specific = "0-10000"  #@param {type: "string"}

#@markdown --- 

#@markdown ### 順番指定リスト
#@markdown 複数人数トレースで、交差後の人物INDEX順番を指定してください。
#@markdown 0F目の立ち位置左から順番に0番目、1番目、と数えます。
#@markdown フォーマット：［＜フレーム番号＞:左から0番目にいる人物のインデックス,左から1番目…］
#@markdown 例）[10:1,0]　…　10F目は、左から1番目の人物、0番目の人物の順番に並べ替えます。
#@markdown [10:1,0][30:0,1]のように、カッコ単位で複数件指定可能です。
reverse_frames = ""  #@param {type: "string"}

#@markdown --- 

#@markdown ### ボーン構造CSVファイル
#@markdown トレース対象モデルのボーン構造CSVファイルのパスを入力して下さい。
#@markdown Googleドライブの "autotrace" フォルダにアップロードしたcsvファイルも読み込めます。
#@markdown その場合、「/gdrive/My Drive/autotrace/[csvファイル名]」のように入力して下さい。
born_model_csv = "born/あにまさ式ミク準標準ボーン.csv"  #@param {type: "string"}

#@markdown --- 

#@markdown ### IKで出力するか
#@markdown 足をIKで出力するか、yes か no を選んで下さい。
#@markdown no を入力した場合、FKで出力します
ik_flag = "yes"  #@param ['yes', 'no']
is_ik = 1 if ik_flag == "yes" else 0

#@markdown ---

#@markdown ### 踵位置補正
#@markdown 踵のY軸補正値を数値(小数可)で入力して下さい。
#@markdown マイナス値を入力すると地面に近付き、プラス値を入力すると地面から遠ざかります。
#@markdown ある程度は自動で補正しますが、補正しきれない場合に、設定して下さい。
heel_position = 0.0  #@param {type: "number"}

#@markdown ---

#@markdown ### センターZ移動倍率
#@markdown センターZ移動に掛ける倍率を数値(小数可)で入力して下さい。
#@markdown 値が小さいほど、センターZ移動の幅が小さくなります。
#@markdown 0を入力した場合、センターZ軸移動を行いません。
center_z_scale = 5.0  #@param {type: "number"}

#@markdown ---

#@markdown ### 滑らかさ
#@markdown モーションの円滑化の度数を指定します
#@markdown 1以上の整数のみを入力して下さい。
#@markdown 度数が大きいほど、円滑化されます。（代わりに動作が小さくなります）
smooth_times = 1  #@param {type: "number"}

#@markdown ---

#@markdown ### 移動間引き量
#@markdown 移動キー（IK・センター）の間引きに使用する移動量を数値(小数可)で指定します
#@markdown 指定された範囲内の移動があった場合に間引きされます。
#@markdown 移動間引き量を0にした場合、間引きを行いません。
threshold_pos = 0.5  #@param {type: "number"}

#@markdown ---

#@markdown ### 回転間引き角度
#@markdown 回転キーの間引きに使用する角度(0～180度まで小数可)を指定します
#@markdown 指定された角度以内の回転があった場合に間引きされます。
threshold_rot = 3  #@param {type: "number"}



!echo 解析を開始するフレーム: "$frame_first"
!echo 映像に映っている最大人数: "$number_people_max"
!echo 反転フレームリスト: "$order_specific"
!echo 順番指定リスト: "$reverse_frames"
!echo ボーン構造CSVファイル: "$born_model_csv"
!echo IKで出力するか: "$ik_flag"
!echo 踵位置補正: "$heel_position"
!echo センターZ移動倍率: "$center_z_scale"
!echo 滑らかさ: "$smooth_times"
!echo 移動間引き量: "$threshold_pos"
!echo 回転間引き角度: "$threshold_rot"

!echo ""
!echo 上記で問題ない場合、次に進んで下さい。

## 自動トレース実行

フォームの入力がすべて完了したら、下のセルを実行してください。

In [0]:
import time
from google.colab import drive
import datetime

start_time = time.time()

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

# 出力フォルダ削除
!rm -r ./output

output_json = "/content/output/json/"
output_openpose_avi = "/content/output/openpose.avi"

! mkdir -p "$output_json"

# 出力用Googleドライブフォルダ
output_path = base_path + "/" + now_str
! mkdir -p "$output_path"

! echo ------------------------------------------
! echo Openpose
! echo ------------------------------------------

# Openpose実行
! cd openpose/ && ./build/examples/openpose/openpose.bin --video "$input_video" --display 0 --model_pose COCO --write_json "$output_json" --write_video "$output_openpose_avi" --number_people_max "$number_people_max" --frame_first "$frame_first"

# Openpose解析結果のコピー
! cp "$output_openpose_avi" "$output_path"

! echo ------------------------------------------
! echo FCRN-DepthPrediction-vmd
! echo ------------------------------------------

! cd FCRN-DepthPrediction-vmd && python tensorflow/predict_video.py --model_path tensorflow/data/NYU_FCRN.ckpt --video_path "$input_video" --json_path "$output_json" --interval 10 --reverse_frames "$reverse_frames" --order_specific "$order_specific" --verbose 1 --now "$now_str"

# 深度結果コピー
depth_dir =  output_json + "_" + now_str + "_depth"
! cp -r "$depth_dir" "$output_path"

for i in range(1, number_people_max+1):
    ! echo ------------------------------------------
    ! echo 3d-pose-baseline-vmd ["$i"]
    ! echo ------------------------------------------
    
    target_dir = output_json + "_" + now_str + "_idx0" + str(i)

    !cd ./3d-pose-baseline-vmd && python src/openpose_3dpose_sandbox_vmd.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 200 --load 4874200 --gif_fps 30 --verbose 1 --openpose "$target_dir" --person_idx 1    
    
    ! echo ------------------------------------------
    ! echo VMD-3d-pose-baseline-multi ["$i"]
    ! echo ------------------------------------------
    
    ! cd ./VMD-3d-pose-baseline-multi && python applications/pos2vmd_multi.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position"
    
    # INDEX別結果コピー
    ! cp -r "$target_dir" "$output_path"

    
elapsed_time = (time.time() - start_time) / 60
    
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■すべての処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo "MMD自動トレース実行結果"

! ls -l "$output_path"